# Spectral Loss search locations, N2 increasing T



In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from fiberamp.fiber.microstruct.bragg import BraggExact
from step_exact import plotlogf
from ngsolve.webgui import Draw
from scipy.optimize import newton

First we load and visualize the spectral loss for N1

# Initialize

In [ ]:
n = 300
wls = np.linspace(1.4e-6, 2e-6, n+1)
outer = 'h2'
nu = 1


In [ ]:
n_air = 1.00027717
n_glass = 1.4388164768221814
ns = [lambda x:n_air, lambda x:n_glass, lambda x:n_air, lambda x : n_glass]

d = 4.0775e-05
ts = [d , 1e-5, d, 1.5e-5]

A = BraggExact(ts=ts, ns=ns, no_mesh=True)

In [ ]:
# Formula for loss spikes from article

ms = np.arange(11,15,1)

n1 = A.ns[0]  # Inner (core) index
n2 = A.ns[1]  # Cladding index

d = 15*2/3*1e-6

ls = (2 * n1 * d / ms * ((n2/n1)**2 - 1)**.5)  # when n2 depends on ls, need solver


# Base N2 Study

In [ ]:
N2betas_base = np.zeros_like(wls, dtype=complex)
k_lows = np.zeros_like(wls)

for i, wl in enumerate(wls):
    print(5 * ' ' +  + 20 * '-' + '  ' + str(i+1) + '/' + str(n+1) + ': ' +
          'wavelength: ' +  str(wls[i]) + '  ' +  20 * '-' +5 * ' '+'\n')
          
    A = BraggExact(ts=ts, maxhs=[100 ,100, 100, 100], wl=wl, ns=ns)

    k_low = A.ks[0] * A.scale
    k_lows[i] = k_low
    guess = np.array(.99995 * k_low)
    imag = 0
    flag = True
    reduce = 0

    while flag:
        try:
            x0 = np.array(guess + imag*1j)
            beta = newton(A.determinant, x0, args=(nu, outer), tol = 1e-17)
            if beta.real > k_low:
                print("Captured wrong mode, retrying.")
                raise RuntimeError
            elif beta.imag > 0:
                print("Positive imaginary part, retrying.")
                imag = -beta.imag
                guess *= 1/.999999
                raise RuntimeError
            else:
                print("Scaled beta: ", beta, ". Residual of determinant: ", 
                      abs(A.determinant(beta, nu, outer)), '\n\n' )
                imag = beta.imag
                flag=False
        except RuntimeError:
            guess *= .999999
            reduce += 1
            print("scaling guess: " + str(reduce), flush=True)
    N2betas_base[i] = beta
    

# Observation on real part of Fundamental Mode

From interactive plots, I observed that at a fixed wavelength, the relative position of the real part of the fundamental mode to k_low does not change appreciably.  Thus, one possible way of filtering out the (small tube) fundamental from other modes is to find this location at each wavelength from the real parts of the base betas we computed above.

In [ ]:
centers = N2betas_base.real

# centers

## Check if this is good

Let's do a run with a larger air gap (still not too large so we know we're getting the right modes) and see if the search centers change much

In [ ]:
N2betas_t1 = np.zeros_like(wls, dtype=complex)

t1 = 1.25
d = 4.0775e-05
ts = [d , 1e-5, t1 * d, 1.5e-5]


for i, wl in enumerate(wls):
    print(5 * ' ' +  + 20 * '-' + '  ' + str(i+1) + '/' + str(n+1) + ': ' +
          'wavelength: ' +  str(wls[i]) + '  ' +  20 * '-' +5 * ' '+'\n')
          
    A = BraggExact(ts=ts, wl=wl, ns=ns, no_mesh=True)

    k_low = A.ks[0] * A.scale
    guess = np.array(.99995 * k_low)
    imag = 0
    flag = True
    reduce = 0

    while flag:
        try:
            x0 = np.array(guess + imag*1j)
            beta = newton(A.determinant, x0, args=(nu, outer), tol = 1e-17)
            if beta.real > k_low:
                print("Captured wrong mode, retrying.")
                raise RuntimeError
            elif beta.imag > 0:
                print("Positive imaginary part, retrying.")
                imag = -beta.imag
                guess *= 1/.999999
                raise RuntimeError
            else:
                print("Scaled beta: ", beta, ". Residual of determinant: ", 
                      abs(A.determinant(beta, nu, outer)), '\n\n' )
                imag = beta.imag
                flag=False
        except RuntimeError:
            guess *= .999999
            reduce += 1
            print("scaling guess: " + str(reduce), flush=True)
    N2betas_t1[i] = beta
    

In [ ]:
centers2 = N2betas_t1.real
np.mean(centers2 / centers)

# See if the centers work directly

This uses them as an intial guess and then downscales


In [ ]:
N2betas_t1_check = np.zeros_like(wls, dtype=complex)

t1 = 1.25
d = 4.0775e-05
ts = [d , 1e-5, t1 * d, 1.5e-5]


for i, wl in enumerate(wls):
    print(5 * ' ' +  + 20 * '-' + '  ' + str(i+1) + '/' + str(n+1) + ': ' +
          'wavelength: ' +  str(wls[i]) + '  ' +  20 * '-' +5 * ' '+'\n')
          
    A = BraggExact(ts=ts, wl=wl, ns=ns, no_mesh=True)

    k_low = A.ks[0] * A.scale
    guess = np.array(centers[i])
    imag = 0
    flag = True
    reduce = 0
#     scaling = 1+1e-6
    scaling = .999999
    while flag:
        try:
            x0 = np.array(guess + imag*1j)
            beta = newton(A.determinant, x0, args=(nu, outer), tol = 1e-17)
            if beta.real > k_low:
                print("Captured wrong mode, retrying.")
                raise RuntimeError
            elif beta.imag > 0:
                print("Positive imaginary part, retrying.")
                imag = -beta.imag
                guess *= 1/scaling
                raise RuntimeError
            else:
                print("Scaled beta: ", beta, ". Residual of determinant: ", 
                      abs(A.determinant(beta, nu, outer)), '\n\n' )
                imag = beta.imag
                flag=False
        except RuntimeError:
            guess *= scaling
            reduce += 1
            print("scaling guess: " + str(reduce), flush=True)
    N2betas_t1_check[i] = beta
    

In [ ]:
%matplotlib inline

fig = plt.figure(figsize=(12,9))

ax = plt.gca()
ax.set_title('\nSpectral Loss Profile Comparison\n', fontsize=18)
ax.set_xlabel('\nwavelength', fontsize=16)
ax.set_ylabel('Confinement Loss\n', fontsize=16)

# ax.plot(wls, -N2betas_base.imag, color='green', linewidth=1.2, label='N2 t=1')
ax.plot(wls, -N2betas_t1.imag, color='blue', linewidth=1.2, label='N2 t=1.5')
ax.plot(wls, -N2betas_t1_check.imag, color='red', linewidth=1.2, label='N2check/ t=1.5')
ax.legend()

ax.set_yscale('log')
m, M = ax.get_ylim()

for l in ls:
    ax.plot([l,l], [m, M],  linewidth=1, linestyle=':')


# Now upscaling


In [ ]:
N2betas_t1_check = np.zeros_like(wls, dtype=complex)

t1 = 1.25
d = 4.0775e-05
ts = [d , 1e-5, t1 * d, 1.5e-5]


for i, wl in enumerate(wls):
    print(5 * ' ' +  + 20 * '-' + '  ' + str(i+1) + '/' + str(n+1) + ': ' +
          'wavelength: ' +  str(wls[i]) + '  ' +  20 * '-' +5 * ' '+'\n')
          
    A = BraggExact(ts=ts, wl=wl, ns=ns, no_mesh=True)

    k_low = A.ks[0] * A.scale
    guess = np.array(centers[i])
    imag = 0
    flag = True
    reduce = 0
    scaling = 1+1e-6
#     scaling = .999999
    while flag:
        try:
            x0 = np.array(guess + imag*1j)
            beta = newton(A.determinant, x0, args=(nu, outer), tol = 1e-17)
            if beta.real > k_low:
                print("Captured wrong mode, retrying.")
                raise RuntimeError
            elif beta.imag > 0:
                print("Positive imaginary part, retrying.")
                imag = -beta.imag
                guess *= 1/scaling
                raise RuntimeError
            else:
                print("Scaled beta: ", beta, ". Residual of determinant: ", 
                      abs(A.determinant(beta, nu, outer)), '\n\n' )
                imag = beta.imag
                flag=False
        except RuntimeError:
            guess *= scaling
            reduce += 1
            print("scaling guess: " + str(reduce), flush=True)
    N2betas_t1_check[i] = beta
    

In [ ]:
%matplotlib inline

fig = plt.figure(figsize=(12,9))

ax = plt.gca()
ax.set_title('\nSpectral Loss Profile Comparison\n', fontsize=18)
ax.set_xlabel('\nwavelength', fontsize=16)
ax.set_ylabel('Confinement Loss\n', fontsize=16)

# ax.plot(wls, -N2betas_base.imag, color='green', linewidth=1.2, label='N2 t=1')
ax.plot(wls, -N2betas_t1.imag, color='blue', linewidth=1.2, label='N2 t=1.5')
ax.plot(wls, -N2betas_t1_check.imag, color='red', linewidth=1.2, label='N2check/ t=1.5')
ax.legend()

ax.set_yscale('log')
m, M = ax.get_ylim()

for l in ls:
    ax.plot([l,l], [m, M],  linewidth=1, linestyle=':')


## Upscaling works better

At least as we initially increase T.  So we'll try this for our procedure.

In [ ]:
%matplotlib inline

fig = plt.figure(figsize=(12,9))

ax = plt.gca()
ax.set_title('\nSpectral Loss Profile Comparison\n', fontsize=18)
ax.set_xlabel('\nwavelength', fontsize=16)
ax.set_ylabel('Confinement Loss\n', fontsize=16)

ax.plot(wls, -N2betas_base.imag, color='green', linewidth=1.2, label='N2 t=1')
ax.plot(wls, -N2betas_t1.imag, color='blue', linewidth=1.2, label='N2 t=1.5')
ax.plot(wls, -N2betas_t1_check.imag, color='red', linewidth=1.2, label='N2check/ t=1.5')
ax.legend()

ax.set_yscale('log')
m, M = ax.get_ylim()

for l in ls:
    ax.plot([l,l], [m, M],  linewidth=1, linestyle=':')
